In [ ]:
# STEP 1

# #This section is base copy from google's sample import of tensorflow, keras, ploting and support libraries

#keras install libraries
!pip install -q keras
!pip install -q tqdm

#openCV is used to process and filter images
!apt-get -qq install -y libsm6 libxext6 && pip install -q -U opencv-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 MB 12.7 MB/s eta 0:00:00


In [ ]:
# STEP 2
# In this step we import the libraries needed to import files from google drive

# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

E: Package 'python-software-properties' has no installation candidate
E: Package 'module-init-tools' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 120874 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.30-1ubuntu1~bpo22.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.30-1ubuntu1~bpo22.04.1) ...
Setting up google-drive-ocamlfuse (0.7.30-1ubuntu1~bpo22.04.1) ...
Processing triggers for man-db (2.10.2-1) ...


In [ ]:
# STEP 3
# Generate auth tokens for Colab

# CLICK THE LINK GENERATED THEN COPY AND PASTE THE KEY INTO THE FIELD AND CLICK ENTER
from google.colab import auth
auth.authenticate_user()

MessageError: ignored

In [ ]:
# STEP 4
# Here we connect to your google drive folder.
# You can download sample puppy and rabbit images using this link: https://drive.google.com/drive/folders/1RcWFv-PDtGK4n1JgkAgL77fB-N0I4oMP?usp=sharing

# WARNING YOU MAY NEED TO RUN THIS CELL MULTIPLE TIMES TO GET A CONNECTION + KEY ESTABLISHED to your google drive.

# CLICK THE 1st LINK GENERATED THEN COPY AND PASTE THE KEY INTO THE FIELD AND CLICK ENTER

# RE-RUN THIS CELL AND CLICK THE 2nd LINK GENERATED THEN COPY AND PASTE THE KEY INTO THE FIELD AND CLICK ENTER

# Generate creds for the Drive FUSE library.
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
vcode = getpass.getpass()

In [ ]:
# STEP 5
# Create a directory and mount Google Drive using that directory.

!mkdir -p drive
!google-drive-ocamlfuse drive

# print 'Files in Drive:'
!ls drive

In [ ]:
import json
print('okay')
json_data=open('drive/train.json').read()
print('cool')
data = json.loads(json_data)
print('done')
print('removed extraneous data')
urls = [obj['url'] for obj in data['images']]

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from itertools import chain, repeat, cycle
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import os
from urllib.request import urlopen



TARGET_SIZE = (256, 256)
BATCH_SIZE = 128

print('meh')
output = []
for dat in data['annotations']:
  temp = [0]*228
  for obj in dat['labelId']:
    temp[int(obj)-1] = 1
  output.append(temp)
depvar = np.array(output)



import threading
print('mmmmmm donuts')
class BatchGenerator:

    def __init__(self, batch_size=32,target_size=(256,256)):
        self.batch_size = batch_size
        self.lock = threading.Lock()
        self.TARGET_SIZE=target_size

    def __iter__(self):
        return self

    def __next__(self):
        with self.lock:
            while True:
                X=[]
                y=[]
                for i in range(32):
                    pic = np.random.randint(0,len(urls))
                    tempnames = depvar[pic].reshape(1,228)
                    try:
                        img_file = urlopen(urls[pic])
                        im = Image.open(img_file)
                    except:
                        output = [1]*(256*256*3)
                        output = np.array(output).reshape(256,256,3).astype('uint8')
                        im = Image.fromarray(output).convert('RGB')
                    im2 = im.resize(self.TARGET_SIZE, Image.ANTIALIAS)
                    im2.thumbnail(self.TARGET_SIZE, Image.ANTIALIAS)
                    #plt.imshow(np.asarray(im2))
                    #plt.show()

                    X_batch = np.asarray(im2).reshape(1,256,256,3)
                    names = depvar[pic]
    #                if np.random.rand()>1:
    #                    zoomed = np.asarray(im2.resize((512,512), Image.ANTIALIAS))
    #                    choice = np.random.randint(0,256)
    #                    #choice = 128
    #                    X_batch = zoomed[choice:choice+256,choice:choice+256,:].reshape(1,256,256,3)
    #                if np.random.rand()>1:
    #                    X_batch = X_batch[:,::-1,:,:]
                    names.reshape(1,228)
                    X_batch = X_batch/255.0
                    X.append(X_batch)
                    y.append(names)
                return np.array(X).reshape(32,256,256,3), np.array(y).reshape(32,228)
    def next(self):
      return self.__next__()

print('check')

train_gen = BatchGenerator(batch_size=1631)
val_gen = BatchGenerator()
#print(train_gen.next())

In [ ]:
from urllib.request import urlopen
from PIL import Image
def convert_pic_to_array(url):
    size = (256, 256)
    try:
        print('pullimg')
        img_file = urlopen(url)
        print('imgopen')
        im = Image.open(img_file)
        print('gothere')
    except:
        output = [0]*(256*256*3)
        output = np.array(output).reshape(1,256,256,3)
        return(output)

    im2 = im.resize(size, Image.ANTIALIAS)
    im2.thumbnail(size, Image.ANTIALIAS)
    output = np.asarray(im2)
    #print(conv_base.predict(output.reshape(1,256,256,3)).shape)
    return(output.reshape(1,256,256,3))
convert_pic_to_array(urls[0])
urlopen(urls[0])

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Reshape
from keras.layers.core import Activation
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import UpSampling2D
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Flatten
from keras.optimizers import SGD,Adamax,Adam
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image
#from keras.applications import VGG19

def discriminator_model():


    model = Sequential()
    model.add(Conv2D(16,
                     (6, 6),
                     strides=(2,2),
                     input_shape=( 256, 256, 3),
                     kernel_initializer='glorot_normal',
                     bias_initializer='zeros',
                     activation='relu'))
    #model.add(Activation('relu'))


    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.1))
    model.add(Conv2D(16, (6, 6), activation='relu',kernel_initializer='glorot_normal'))
    model.add(Conv2D(16, (6, 6), activation='relu',kernel_initializer='glorot_normal'))
    model.add(Conv2D(16, (6, 6), activation='relu',kernel_initializer='glorot_normal'))
    model.add(Conv2D(64, (5, 5), activation='relu',kernel_initializer='glorot_normal'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.1))
    model.add(Conv2D(64, (5, 5), activation='relu',kernel_initializer='glorot_normal'))
    model.add(Conv2D(64, (5, 5), activation='relu',kernel_initializer='glorot_normal'))
    model.add(Conv2D(64, (5, 5), activation='relu',kernel_initializer='glorot_normal'))
    model.add(Conv2D(256, (3, 3), activation='relu',kernel_initializer='glorot_normal'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.1))
    model.add(Conv2D(128, (3, 3), activation='relu',kernel_initializer='glorot_normal'))
    #model.add(Conv2D(128, (3, 3), activation='relu',kernel_initializer='glorot_normal'))

   # model.add(Conv2D(256, (3, 3), activation='relu',kernel_initializer='glorot_normal'))
    #model.add(BatchNormalization())
    #model.add(Conv2D(512, (5, 5), activation='relu',kernel_initializer='glorot_normal'))
    #model.add(BatchNormalization())
    #model.add(Conv2D(1024, (5, 5), activation='relu',kernel_initializer='glorot_normal'))
  #  model.add(BatchNormalization())
    #model.add(Conv2D(2048, (5, 5), activation='relu',kernel_initializer='glorot_normal'))
    #model.add(BatchNormalization())
    #model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(1024))
    model.add(Dense(228,activation='sigmoid',kernel_initializer='glorot_normal'))
    return model
d = discriminator_model()
d_optim = Adam(clipnorm=0.10)
d.compile(loss='categorical_crossentropy', optimizer=d_optim)

In [ ]:
from keras.models import load_model
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Reshape
from keras.layers.core import Activation
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import UpSampling2D
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Flatten
from keras.optimizers import SGD,Adamax
d = load_model('drive/model.h5')
d.load_weights('drive/model_weights.h5')
print(d.summary())

In [ ]:
from keras.optimizers import RMSprop, Adam
#from keras.callbacks import ModelCheckpoint
d_optim = Adam(lr=0.00001, clipnorm=0.9)
d.compile(loss='categorical_crossentropy', optimizer=d_optim)
#d_optim = RMSprop(lr=0.00001, clipnorm=0.5)
#d.compile(loss='categorical_crossentropy', optimizer=d_optim)
#cweights = {}
#cw = 1/(np.sum(depvar,axis=0)/np.sum(depvar))
#for i in range(len(depvar[0])):
#    cweights[i] = cw[i]


train_gen = BatchGenerator(batch_size=32)
#for i in range(1000):
x,y = next(train_gen)
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint('drive/model_weights.h5')
callbacks_list = [checkpoint]
#if i%4==0:
d.save_weights('drive/model_weights.h5')
history = d.fit_generator(train_gen,steps_per_epoch=250,epochs=110,callbacks=callbacks_list,workers=5,use_multiprocessing=True)#, validation_data=val_gen,validation_steps=1)
#else:
#d.fit(x,y,epochs=10)
d.save('drive/model.h5')
d.save_weights('drive/model_weights.h5')
test = d.predict(convert_pic_to_array(urls[0]))
li = {}
for i in range(len(test[0])):
    li[i+1]=test[0,i]
print(sorted(li.items(), key=lambda x: x[1], reverse=True))
print(data['annotations'][0]['labelId'])

In [ ]:
from keras import backend as K
x1,y1 = next(train_gen)
inp = d.input                                           # input placeholder                                       # input placeholder
outputs = d.layers[0].output
functors = K.function([inp], [outputs])
s = functors([x1])[0].shape
print(s)
arr = functors([x1])[0].reshape(s)

p = ((255./(np.max(arr[5])-np.min(arr[5])))*(arr[5]-np.min(arr[5])))
#p=255.0*arr[i,:,:,2]
print(p.shape)
fig, ax=plt.subplots(dpi=300)
ax.imshow(p.reshape(s[1], s[2]*s[3],order='F'))
ax.grid (False)
plt.show()
plt.imshow(x1[5])
plt.grid(False)
plt.show()